In [868]:
import pandas as pd
import numpy as np


In [869]:
diagnosis = pd.read_csv('../data/DIAGNOSIS.csv')
demographics = pd.read_csv('../data/DEMOGRAPHICS.csv')
apoe = pd.read_csv('../data/APOE.csv')
print(demographics.columns)

apoe['APOE41'] = apoe['GENOTYPE'].apply(lambda x: 1 if x == '3/4' else 0)
apoe = apoe[['RID', 'APOE41']]
demographics = demographics[['RID', 'PTDOB', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT']]
demographics = demographics.drop_duplicates(subset=['RID'])
diagnosis = diagnosis[['RID', 'VISCODE2', 'DIAGNOSIS', 'EXAMDATE']]

merged_df = pd.merge(demographics, apoe, on='RID', how='inner')
merged_df = pd.merge(merged_df, diagnosis, on='RID', how='right')
merged_df = merged_df.dropna()
merged_df['PTDOB'] = pd.to_datetime(merged_df['PTDOB'], errors='coerce')
merged_df['EXAMDATE'] = pd.to_datetime(merged_df['EXAMDATE'], errors='coerce')
merged_df['AGE'] = (merged_df['EXAMDATE'] - merged_df['PTDOB']).dt.days / 365.25

merged_df = merged_df.sort_values(by=['RID', 'EXAMDATE'])

Index(['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT',
       'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR'

/var/folders/g3/2dtjm6cd09v21m_fw6x_ggx00000gp/T/ipykernel_6532/2976151376.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['PTDOB'] = pd.to_datetime(merged_df['PTDOB'], errors='coerce')


In [350]:
demographics

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT
0,2,04/1931,1.0,16.0,5
1,1,12/1944,2.0,18.0,-4
2,3,05/1924,1.0,18.0,5
3,4,01/1938,1.0,10.0,5
4,5,12/1931,1.0,16.0,5
...,...,...,...,...,...
6187,10862,03/1957,1.0,14.0,1|5
6188,10850,01/1943,1.0,18.0,5
6191,10846,11/1953,1.0,13.0,5
6193,10684,01/1956,1.0,14.0,4


In [155]:
ucsf1 = pd.read_csv('../data/UCSF_1.csv')
ucsf2 = pd.read_csv('../data/UCSF_6.csv')
ucsf3 = pd.read_csv('../data/UCSF7.csv')
ucsf4 = pd.read_csv('../data/UCSF71.csv')

common_cols = list(set(ucsf1.columns) & set(ucsf2.columns) & set(ucsf3.columns) & set(ucsf4.columns))

# Optionally: keep them in the same order as in ucsf4
common_cols = [col for col in ucsf4.columns if col in common_cols]

ucsf1 = ucsf1[common_cols]
ucsf2 = ucsf2[common_cols]
ucsf3 = ucsf3[common_cols]
ucsf4 = ucsf4[common_cols]


def get_keys(df):
    return set(df['IMAGEUID'])

keys_combined = get_keys(ucsf4)
keys_expected = get_keys(ucsf1) | get_keys(ucsf2) | get_keys(ucsf3)

# Check if all keys in keys_combined are in keys_expected
missing_keys = keys_expected - keys_combined

# add missing keys to ucsf4
for key in missing_keys:
    rid = key
    # Find the corresponding row in ucsf1, ucsf2, or ucsf3
    for df in [ucsf1, ucsf2, ucsf3]:
        if key in get_keys(df):
            missing_row = df[(df['IMAGEUID'] == rid)]
            if not missing_row.empty and key not in get_keys(ucsf4):
                # Append the row to ucsf4
                ucsf4 = pd.concat([ucsf4, missing_row], ignore_index=True)
                # remove the key from missing_keys

ucsf4

/var/folders/g3/2dtjm6cd09v21m_fw6x_ggx00000gp/T/ipykernel_6532/1764264955.py:4: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  ucsf4 = pd.read_csv('../data/UCSF71.csv')


,RID,VISCODE,IMAGEUID,EXAMDATE,RUNDATE,STATUS,OVERALLQC,TEMPQC,FRONTQC,PARQC,...,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV,update_stamp
0,4213,init,888008,2017-08-14,2022-07-23,partial,NaN,NaN,NaN,NaN,...,6240.0,1671.0,2.849,0.958,9704.0,3196.0,2.498,0.744,1866.4,2024-12-02 08:27:54.0
1,4453,v02,281425,2012-01-18,2022-10-05,partial,NaN,NaN,NaN,NaN,...,5339.0,2038.0,2.376,0.701,9877.0,3248.0,2.651,0.578,1707.8,2024-12-02 08:27:54.0
2,4104,v11,322667,2012-07-11,2022-09-26,partial,NaN,NaN,NaN,NaN,...,5424.0,1879.0,2.527,0.706,10780.0,3032.0,2.903,0.580,3493.8,2024-12-02 08:27:54.0
3,2153,v21,417092,2014-03-03,2022-08-14,partial,NaN,NaN,NaN,NaN,...,5992.0,1883.0,2.691,0.891,9898.0,2940.0,2.801,0.603,2618.2,2024-12-02 08:27:54.0
4,4303,v04,288921,2012-02-27,2022-10-05,partial,NaN,NaN,NaN,NaN,...,4878.0,1885.0,2.273,0.706,9481.0,2861.0,2.774,0.540,1528.3,2024-12-02 08:27:54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13080,4447,v02,280563,2012-01-17,2018-06-05,partial,NaN,NaN,NaN,NaN,...,4553.0,1848.0,2.374,0.717,9240.0,2700.0,2.840,0.711,1448.0,2019-11-14 14:34:05.0
13081,4020,v02,233461,2011-05-03,2012-02-10,partial,Pass,Pass,Pass,Pass,...,4828.0,1940.0,2.206,0.697,11934.0,3559.0,2.819,0.713,2098.0,2019-11-14 14:34:03.0
13082,2026,nv,223220,2010-07-26,2012-02-10,partial,Partial,Fail,Pass,Pass,...,4034.0,1533.0,2.355,0.995,8326.0,2674.0,2.692,0.615,1957.0,2019-11-14 14:34:01.0
13083,2026,nv,223226,2010-10-28,2012-02-10,partial,Partial,Fail,Pass,Pass,...,4159.0,1414.0,2.562,1.139,8234.0,2747.0,2.647,0.572,1758.0,2019-11-14 14:34:01.0


In [202]:
mri = ucsf4
# # drop rows where more than 20% of the values are missing
mri = mri[mri.isnull().sum(axis=1) < 25]


mri = mri.dropna(subset=['EXAMDATE', 'RID']).sort_values(by=['EXAMDATE'])
print(mri.shape)

qc_cols = ['TEMPQC', 'FRONTQC', 'PARQC', 'INSULAQC', 'OCCQC', 'BGQC', 'CWMQC', 'VENTQC']
print(mri[qc_cols].isnull().sum())
# filtered_df = mri[mri[qc_cols].eq('Pass').all(axis=1)]
# mri = filtered_df

mri['EXAMDATE'] = pd.to_datetime(mri['EXAMDATE'])

merged_mri_mci = mri.drop(columns=[ 'IMAGEUID', 'RUNDATE', 'STATUS',  'ST8SV', 'ST68SV', 'ST125SV', 'ST66SV'])
mri_mci_selected = merged_mri_mci
selected_columns = [col for col in mri_mci_selected.columns if 'SUVR' in col or 'SV' in col or 'CV' in col or 'RID' in col or 'EXAMDATE' in col or 'VISCODE' in col]
mri_mci = mri_mci_selected[selected_columns]

datadic = pd.read_csv('/Users/nataliaglazman/Desktop/PhD/work/project/data/dictionaries/datadic2.csv')
datadic['Grouping'] = datadic['Broader Region'].str.split('(').str[0]
datadic['Grouping'] = datadic['Grouping'].str.rstrip()

# get MRI dictionary
dic = pd.read_csv('/Users/nataliaglazman/Desktop/PhD/work/project/data/dictionaries/dictionary_mri.csv', header=None)
dic['Brain Region'] = dic.iloc[:,1].str.split(' ').str[-1]

# drop all rows with 'Thickness' or 'Surface Area' in the second column
dic = dic[~dic.iloc[:,1].str.contains('Thickness|Surface Area')]

datadic['Brain Region'] = datadic['Brain Region'].str.split('(').str[0]
datadic['Brain Region'] = datadic['Brain Region'].str.replace(' ', '')
new = pd.merge(dic, datadic, on='Brain Region', how='right')
new = new.drop(columns=[1])
new = new.drop(columns=['Broader Region', 'Main Functions'])
mapping = dict(zip(new[0], new['Grouping']))
unique_groups = new['Grouping'].unique()

df_grouped = pd.DataFrame(index=mri_mci.index)
df_grouped = mri_mci[['RID', 'EXAMDATE', 'VISCODE']].copy()
print(df_grouped.shape)

for grp in unique_groups:
    # Find all columns in df_main belonging to this group
    group_cols = [col for col in mri_mci.columns if mapping.get(col) == grp]
    
    # Sum across these columns (axis=1 sums across columns to produce a single Series)
    df_grouped[grp] = mri_mci[group_cols].sum(axis=1)

for i in df_grouped.columns:
    # check if the dtype of the column is float
    if df_grouped[i].dtype == 'float64':
        # check if the sum of the column is 0
        if df_grouped[i].sum() == 0:
            df_grouped = df_grouped.drop(columns=[i])

df_grouped = df_grouped[['RID','EXAMDATE', 'VISCODE', 'Intracranial Volume','Hippocampus',  'Amygdala', 'Temporal Lobe', 'Parietal Lobe', 'Frontal Lobe', 'Occipital Lobe', 'Cingulate Cortex', 'Insular Cortex', 'Basal Ganglia', 'Thalamus']]

(12538, 330)
TEMPQC      11185
FRONTQC     11185
PARQC       11185
INSULAQC    11185
OCCQC       11185
BGQC        11185
CWMQC       11185
VENTQC      11185
dtype: int64
(12538, 3)


In [951]:
merged = pd.merge(merged_df, df_grouped, on='RID', suffixes=('_x', '_y'))

# Compute absolute date difference
merged['date_diff'] = (merged['EXAMDATE_x'] - merged['EXAMDATE_y']).abs()

# Filter to matches within ±60 days
within_2_months = merged[merged['date_diff'] <= pd.Timedelta(days=100)]
# For each df1 row (by ID and date_df1), pick the df2 row with the closest date
closest_match = within_2_months.sort_values('date_diff').drop_duplicates(['RID', 'EXAMDATE_y'])

In [952]:
closest_match

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,VISCODE2,DIAGNOSIS,EXAMDATE_x,AGE,...,Amygdala,Temporal Lobe,Parietal Lobe,Frontal Lobe,Occipital Lobe,Cingulate Cortex,Insular Cortex,Basal Ganglia,Thalamus,date_diff
89470,10612,1963-12-01,1.0,16.0,5,0.0,sc,1.0,2025-04-29,61.409993,...,3454.9,75194.0,102074.0,144323.0,50973.0,10092.0,13779.0,19053.6,12847.7,0 days
18098,514,1925-08-01,1.0,20.0,5,0.0,m48,3.0,2010-06-09,84.854209,...,2942.4,68672.0,108719.0,138088.0,44281.0,7142.0,12438.0,30719.2,15213.4,0 days
18108,514,1925-08-01,1.0,20.0,5,0.0,m60,3.0,2011-06-08,85.850787,...,2889.8,68258.0,106037.0,138484.0,42102.0,8490.0,12798.0,34980.1,14156.9,0 days
18122,516,1918-09-01,1.0,15.0,5,0.0,m12,1.0,2007-06-08,88.766598,...,2765.4,71969.0,92123.0,129788.0,37611.0,9116.0,15064.0,26338.5,14296.0,0 days
18128,516,1918-09-01,1.0,15.0,5,0.0,m24,1.0,2008-06-04,89.757700,...,2601.5,71024.0,92462.0,129729.0,37343.0,8880.0,15136.0,26434.1,14443.3,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61332,4292,1940-11-01,1.0,15.0,5,0.0,m06,1.0,2012-07-20,71.715264,...,3044.1,69019.0,98098.0,129836.0,44025.0,10613.0,12217.0,18303.5,12917.1,100 days
83400,6057,1945-12-01,2.0,20.0,6,0.0,m24,1.0,2020-06-02,74.502396,...,3150.2,69246.0,101681.0,129636.0,47551.0,9898.0,12896.0,22616.3,12602.3,100 days
48494,2220,1928-11-01,1.0,20.0,5,0.0,m90,2.0,2018-10-24,89.976728,...,3401.0,67881.0,93817.0,144492.0,53916.0,10726.0,12517.0,21969.9,9980.3,100 days
33646,1062,1924-11-01,1.0,12.0,5,1.0,bl,3.0,2007-02-21,82.305270,...,1942.2,72024.0,108243.0,140396.0,41085.0,11627.0,13205.0,19228.9,12096.2,100 days


In [953]:

closest_match[['RID', 'EXAMDATE_y','date_diff', 'VISCODE', 'Hippocampus', 'Amygdala']].sort_values(by=['RID', 'EXAMDATE_y'])

,RID,EXAMDATE_y,date_diff,VISCODE,Hippocampus,Amygdala
0,2,2005-08-26,34 days,sc,8211.8,3303.0
10,3,2005-09-01,29 days,sc,5989.4,2720.9
15,3,2006-03-13,0 days,m06,5904.9,2297.2
20,3,2006-09-12,1 days,m12,5811.5,2419.6
25,3,2007-09-12,0 days,m24,6397.8,2743.7
...,...,...,...,...,...,...
89462,10484,2025-02-20,8 days,4_sc,7912.7,2628.1
89464,10486,2025-02-19,7 days,4_sc,6939.4,2732.4
89466,10487,2025-02-19,7 days,4_sc,8419.4,3328.1
89468,10513,2025-03-02,10 days,4_sc,6901.1,2565.1


In [954]:
blood = pd.read_csv('/Users/nataliaglazman/Desktop/PhD/work/project/data/ADNI/UPenn Plasma Quanterix Apr 25 2025.csv')
blood = pd.read_csv('../data/BLOOD.csv')

In [955]:
csf = pd.read_csv('../data/CSF.csv')
csf = csf[['RID', 'EXAMDATE', 'ABETA40', 'ABETA42', 'TAU', 'PTAU']]

In [956]:
closest_match.columns

Index(['RID', 'PTDOB', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT', 'APOE41',
       'VISCODE2', 'DIAGNOSIS', 'EXAMDATE_x', 'AGE', 'EXAMDATE_y', 'VISCODE',
       'Intracranial Volume', 'Hippocampus', 'Amygdala', 'Temporal Lobe',
       'Parietal Lobe', 'Frontal Lobe', 'Occipital Lobe', 'Cingulate Cortex',
       'Insular Cortex', 'Basal Ganglia', 'Thalamus', 'date_diff'],
      dtype='object')

In [957]:
# merged = pd.merge(closest_match, csf, on="RID", suffixes=("_c", "_b"))

# # Convert to datetime
# merged["EXAMDATE_c"] = pd.to_datetime(merged["EXAMDATE_x"])
# merged["EXAMDATE_b"] = pd.to_datetime(merged["EXAMDATE"])

# # Compute absolute date difference
# merged["date_diff"] = (merged["EXAMDATE_c"] - merged["EXAMDATE_b"]).abs()

# # Filter to matches within ±60 days
# within_2_months = merged[merged["date_diff"] <= pd.Timedelta(days=100)]

# # For each (RID, EXAMDATE_c), pick the closest EXAMDATE_b
# closest_match = within_2_months.sort_values("date_diff").drop_duplicates(
#     subset=["RID", "EXAMDATE_b"]
# )


In [958]:
closest_match.sort_values(by=['RID', 'EXAMDATE_c'])

KeyError: 'EXAMDATE_c'

In [908]:
# find duplicates of Hippocampus
closest_match['Hippocampus'].duplicated().sum()

np.int64(79)

In [909]:
closest_match['VISCODE2'].value_counts()

VISCODE2
bl      1084
m24      456
sc       416
m12      299
m48      185
m36       78
m72       50
m60       42
m96       23
m66       23
m84       21
m30       17
m06       16
m78       16
m90       12
m102      11
m108      10
m132       8
m54        7
m120       6
m114       3
m156       3
m18        3
m126       2
m42        2
m138       2
m180       1
Name: count, dtype: int64

In [959]:
merged = pd.merge(closest_match, blood, on="RID", suffixes=("_g", "_d"))
print(merged.columns)
# Convert to datetime
merged["EXAMDATE_g"] = pd.to_datetime(merged["EXAMDATE_x"])
merged["EXAMDATE_d"] = pd.to_datetime(merged["EXAMDATE"])

# Compute absolute date difference
merged["date_diff"] = (merged["EXAMDATE_g"] - merged["EXAMDATE_d"]).abs()

# Filter to matches within ±60 days
within_2_months = merged[merged["date_diff"] <= pd.Timedelta(days=100)]

# For each (RID, EXAMDATE_c), pick the closest EXAMDATE_b
closest_match = within_2_months.sort_values("date_diff").drop_duplicates(
    subset=["RID", "EXAMDATE_d"]
)

print(closest_match)

Index(['RID', 'PTDOB', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT', 'APOE41',
       'VISCODE2_g', 'DIAGNOSIS', 'EXAMDATE_x', 'AGE', 'EXAMDATE_y',
       'VISCODE_g', 'Intracranial Volume', 'Hippocampus', 'Amygdala',
       'Temporal Lobe', 'Parietal Lobe', 'Frontal Lobe', 'Occipital Lobe',
       'Cingulate Cortex', 'Insular Cortex', 'Basal Ganglia', 'Thalamus',
       'date_diff', 'PHASE', 'PTID', 'VISCODE_d', 'VISCODE2_d', 'EXAMDATE',
       'Primary', 'Additive', 'pT217_F', 'AB42_F', 'AB40_F', 'AB42_AB40_F',
       'pT217_AB42_F', 'NfL_Q', 'GFAP_Q', 'update_stamp'],
      dtype='object')
       RID      PTDOB  PTGENDER  PTEDUCAT PTRACCAT  APOE41 VISCODE2_g  \
3      514 1925-08-01       1.0      20.0        5     0.0        m60   
4452   751 1935-08-01       1.0      16.0        5     0.0         bl   
2454  6327 1941-02-01       2.0      16.0        5     1.0         bl   
4485  4275 1938-12-01       1.0      18.0        5     0.0        m06   
2381   256 1936-01-01       1.0      20.0    

In [986]:
# find number of duplicated RIDs in blood
closest_match['RID'].value_counts()

RID
4526     2
1169     2
6189     2
6995     2
4962     2
        ..
6069     1
6072     1
6075     1
6080     1
10457    1
Name: count, Length: 807, dtype: int64

In [960]:
# find duplicated RIDs
duplicated_rids = closest_match[closest_match.duplicated(subset=['RID'], keep=False)]
duplicated_rids

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,VISCODE2_g,DIAGNOSIS,EXAMDATE_x,AGE,...,pT217_F,AB42_F,AB40_F,AB42_AB40_F,pT217_AB42_F,NfL_Q,GFAP_Q,update_stamp,EXAMDATE_g,EXAMDATE_d
3,514,1925-08-01,1.0,20.0,5,0.0,m60,3.0,2011-06-08,85.850787,...,0.111,36.19,374.31,0.096685,0.003067,47.8,282.8,2024-12-18 09:58:07.0,2011-06-08,2011-06-08
2454,6327,1941-02-01,2.0,16.0,5,1.0,bl,1.0,2018-05-21,77.297741,...,0.319,21.54,228.29,0.094400,0.014810,20.5,178.3,2024-12-18 09:58:07.0,2018-05-21,2018-05-21
2381,256,1936-01-01,1.0,20.0,5,0.0,m72,3.0,2012-03-19,76.213552,...,1.084,27.64,317.02,0.087187,0.039219,44.3,323.8,2024-12-18 09:58:07.0,2012-03-19,2012-03-19
4546,303,1921-12-01,1.0,16.0,5,1.0,m132,1.0,2017-04-12,95.362081,...,0.118,29.64,325.19,0.091100,0.003980,35.1,211.5,2024-12-18 09:58:07.0,2017-04-12,2017-04-12
4575,702,1921-07-01,1.0,16.0,5,1.0,bl,2.0,2006-08-23,85.144422,...,0.381,-4.00,-4.00,-4.000000,-4.000000,38.0,205.2,2024-12-18 09:58:07.0,2006-08-23,2006-08-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5507,492,1918-12-01,1.0,18.0,5,1.0,bl,3.0,2006-09-01,87.750856,...,0.852,26.69,375.42,0.071094,0.031922,25.9,358.1,2024-12-18 09:58:07.0,2006-09-01,2006-05-31
4253,6995,1963-01-01,2.0,18.0,2,0.0,sc,1.0,2021-08-30,58.661191,...,0.079,24.65,237.49,0.103800,0.003200,10.7,166.3,2024-12-18 09:58:07.0,2021-08-30,2021-12-02
5508,492,1918-12-01,1.0,18.0,5,1.0,bl,3.0,2006-09-01,87.750856,...,0.673,29.85,386.98,0.077136,0.022546,27.0,366.0,2024-12-18 09:58:07.0,2006-09-01,2006-12-07
3195,6145,1947-08-01,2.0,16.0,5,0.0,sc,1.0,2017-12-06,70.349076,...,0.128,28.59,283.28,0.100900,0.004480,15.4,125.7,2024-12-18 09:58:07.0,2017-12-06,2018-03-13


In [961]:
adas = pd.read_csv('../data/ADAS.csv')
adas['VISDATE'] = pd.to_datetime(adas['VISDATE'], errors='coerce')
adas = adas[['RID',  'TOTAL13', 'VISDATE']]
adas = adas.dropna(subset=['VISDATE', 'RID']).sort_values(by=['VISDATE'])
neurobat = pd.read_csv('../data/NEUROBAT.csv')
neurobat['VISDATE'] = pd.to_datetime(neurobat['VISDATE'], errors='coerce')
neurobat = neurobat[['RID', 'TRABSCOR', 'VISDATE']]
neurobat = neurobat.dropna(subset=['VISDATE', 'RID']).sort_values(by=['VISDATE'])
mmse = pd.read_csv('../data/MMSE.csv')
mmse['VISDATE'] = pd.to_datetime(mmse['VISDATE'], errors='coerce')
mmse = mmse[['RID', 'MMSCORE', 'VISDATE']]
mmse = mmse.dropna(subset=['VISDATE', 'RID']).sort_values(by=['VISDATE'])

/var/folders/g3/2dtjm6cd09v21m_fw6x_ggx00000gp/T/ipykernel_6532/54356079.py:5: DtypeWarning: Columns (79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  neurobat = pd.read_csv('../data/NEUROBAT.csv')


In [962]:
merged = pd.merge(closest_match, adas, on="RID", suffixes=("_c", "_m"))

print(merged.columns)
merged["VISDATE"] = pd.to_datetime(merged["VISDATE"])

# Compute absolute date difference
merged["date_diff"] = (merged["EXAMDATE_x"] - merged["VISDATE"]).abs()

# Filter to matches within ±60 days
within_2_months = merged[merged["date_diff"] <= pd.Timedelta(days=100)]

# For each (RID, EXAMDATE_c), pick the closest EXAMDATE_b
closest_match = within_2_months.sort_values("date_diff").drop_duplicates(
    subset=["RID", "VISDATE"])

Index(['RID', 'PTDOB', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT', 'APOE41',
       'VISCODE2_g', 'DIAGNOSIS', 'EXAMDATE_x', 'AGE', 'EXAMDATE_y',
       'VISCODE_g', 'Intracranial Volume', 'Hippocampus', 'Amygdala',
       'Temporal Lobe', 'Parietal Lobe', 'Frontal Lobe', 'Occipital Lobe',
       'Cingulate Cortex', 'Insular Cortex', 'Basal Ganglia', 'Thalamus',
       'date_diff', 'PHASE', 'PTID', 'VISCODE_d', 'VISCODE2_d', 'EXAMDATE',
       'Primary', 'Additive', 'pT217_F', 'AB42_F', 'AB40_F', 'AB42_AB40_F',
       'pT217_AB42_F', 'NfL_Q', 'GFAP_Q', 'update_stamp', 'EXAMDATE_g',
       'EXAMDATE_d', 'TOTAL13', 'VISDATE'],
      dtype='object')


In [963]:
duplicated_rids = closest_match[closest_match.duplicated(subset=['RID'], keep=False)]
duplicated_rids

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,VISCODE2_g,DIAGNOSIS,EXAMDATE_x,AGE,...,AB40_F,AB42_AB40_F,pT217_AB42_F,NfL_Q,GFAP_Q,update_stamp,EXAMDATE_g,EXAMDATE_d,TOTAL13,VISDATE
7,514,1925-08-01,1.0,20.0,5,0.0,m60,3.0,2011-06-08,85.850787,...,374.31,0.096685,0.003067,47.8,282.8,2024-12-18 09:58:07.0,2011-06-08,2011-06-08,11.33,2011-06-08
1088,6446,1957-01-01,1.0,12.0,5,0.0,m72,2.0,2024-07-10,67.520876,...,417.73,0.088600,0.003220,14.9,85.8,2025-08-06 10:07:33.0,2024-07-10,2024-07-10,15.33,2024-07-10
1091,6891,1942-06-01,1.0,15.0,5,1.0,m12,3.0,2021-11-01,79.419576,...,363.16,0.081500,0.020760,34.6,201.0,2024-12-18 09:58:07.0,2021-11-01,2021-11-01,31.33,2021-11-01
1101,1308,1927-06-01,1.0,14.0,5,1.0,m24,3.0,2009-02-25,81.738535,...,344.30,0.080627,0.014553,32.7,239.3,2024-12-18 09:58:07.0,2009-02-25,2009-02-25,35.67,2009-02-25
1130,78,1929-12-01,2.0,18.0,5,1.0,m18,3.0,2007-07-24,77.642710,...,305.59,0.089106,0.046419,47.7,504.1,2024-12-18 09:58:07.0,2007-07-24,2007-07-24,72.00,2007-07-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,4765,1936-06-01,1.0,16.0,5,1.0,m60,3.0,2017-08-29,81.242984,...,-4.00,-4.000000,-4.000000,-5.0,-5.0,2024-12-18 09:58:07.0,2017-08-29,2017-06-21,46.67,2017-06-02
4300,492,1918-12-01,1.0,18.0,5,1.0,bl,3.0,2006-09-01,87.750856,...,375.42,0.071094,0.031922,25.9,358.1,2024-12-18 09:58:07.0,2006-09-01,2006-05-31,30.00,2006-05-31
4303,6995,1963-01-01,2.0,18.0,2,0.0,sc,1.0,2021-08-30,58.661191,...,237.49,0.103800,0.003200,10.7,166.3,2024-12-18 09:58:07.0,2021-08-30,2021-12-02,10.33,2021-12-02
4307,6145,1947-08-01,2.0,16.0,5,0.0,sc,1.0,2017-12-06,70.349076,...,283.28,0.100900,0.004480,15.4,125.7,2024-12-18 09:58:07.0,2017-12-06,2018-03-13,6.67,2018-03-13


In [964]:
merged = pd.merge(closest_match, mmse, on="RID", suffixes=("_c", "_m"))

merged["VISDATE_m"] = pd.to_datetime(merged["VISDATE_m"])

# Compute absolute date difference
merged["date_diff"] = (merged["EXAMDATE_x"] - merged["VISDATE_m"]).abs()

# Filter to matches within ±60 days
within_2_months = merged[merged["date_diff"] <= pd.Timedelta(days=100)]

# For each (RID, EXAMDATE_c), pick the closest EXAMDATE_b
closest_match = within_2_months.sort_values("date_diff").drop_duplicates(
    subset=["RID", "VISDATE_m"])

In [965]:
duplicated_rids = closest_match[closest_match.duplicated(subset=['RID'], keep=False)]
duplicated_rids

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,VISCODE2_g,DIAGNOSIS,EXAMDATE_x,AGE,...,pT217_AB42_F,NfL_Q,GFAP_Q,update_stamp,EXAMDATE_g,EXAMDATE_d,TOTAL13,VISDATE_c,MMSCORE,VISDATE_m
7,514,1925-08-01,1.0,20.0,5,0.0,m60,3.0,2011-06-08,85.850787,...,0.003067,47.80,282.80,2024-12-18 09:58:07.0,2011-06-08,2011-06-08,11.33,2011-06-08,28.0,2011-06-08
1320,723,1927-06-01,1.0,12.0,5,0.0,m36,3.0,2009-08-05,82.179329,...,0.010422,278.00,274.00,2024-12-18 09:58:07.0,2009-08-05,2009-08-05,28.33,2009-08-05,23.0,2009-08-05
3744,7055,1952-04-01,1.0,16.0,4,0.0,sc,1.0,2022-03-14,69.949350,...,0.006930,15.50,138.40,2024-12-18 09:58:07.0,2022-03-14,2022-04-11,6.67,2022-04-19,27.0,2022-03-14
1373,256,1936-01-01,1.0,20.0,5,0.0,m72,3.0,2012-03-19,76.213552,...,0.039219,44.30,323.80,2024-12-18 09:58:07.0,2012-03-19,2012-03-19,61.00,2012-03-19,12.0,2012-03-19
1470,6931,1948-07-01,1.0,20.0,5,0.0,m36,1.0,2024-08-07,76.101300,...,0.003220,10.90,172.30,2025-08-06 10:07:33.0,2024-08-07,2024-08-07,14.33,2024-08-07,29.0,2024-08-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4222,896,1929-11-01,1.0,16.0,5,0.0,m150,3.0,2019-06-04,89.587953,...,0.008160,46.70,280.60,2024-12-18 09:58:07.0,2019-06-04,2019-04-09,16.67,2019-03-20,27.0,2019-03-20
4053,4546,1941-02-01,1.0,20.0,5,1.0,bl,3.0,2012-05-03,71.249829,...,0.019507,16.60,238.60,2024-12-18 09:58:07.0,2012-05-03,2012-04-16,27.67,2012-03-12,25.0,2012-02-13
4248,6929,1958-03-01,2.0,14.0,5,0.0,m36,2.0,2024-08-09,66.442163,...,0.003050,7.97,51.36,2025-08-06 10:07:33.0,2024-08-09,2024-07-31,9.67,2024-05-21,30.0,2024-05-21
3762,4071,1926-11-01,1.0,13.0,5,0.0,bl,1.0,2011-08-23,84.807666,...,0.004670,19.20,186.20,2024-12-18 09:58:07.0,2011-08-23,2011-08-22,17.67,2011-07-18,24.0,2011-06-01


In [966]:
closest_match

,RID,PTDOB,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,VISCODE2_g,DIAGNOSIS,EXAMDATE_x,AGE,...,pT217_AB42_F,NfL_Q,GFAP_Q,update_stamp,EXAMDATE_g,EXAMDATE_d,TOTAL13,VISDATE_c,MMSCORE,VISDATE_m
7,514,1925-08-01,1.0,20.0,5,0.0,m60,3.0,2011-06-08,85.850787,...,0.003067,47.80,282.8,2024-12-18 09:58:07.0,2011-06-08,2011-06-08,11.33,2011-06-08,28.0,2011-06-08
3747,10391,1967-09-01,2.0,16.0,5,0.0,sc,1.0,2024-12-03,57.256674,...,0.003220,10.50,118.9,2025-08-06 10:07:33.0,2024-12-03,2025-01-15,3.33,2025-01-08,29.0,2024-12-03
3748,10437,1956-02-01,1.0,18.0,4,1.0,sc,2.0,2025-01-13,68.950034,...,0.047840,21.30,194.7,2025-08-06 10:07:33.0,2025-01-13,2025-02-11,20.67,2025-02-18,24.0,2025-01-13
1335,7041,1948-10-01,2.0,20.0,5,1.0,sc,2.0,2022-02-09,73.357974,...,0.003400,15.30,137.6,2024-12-18 09:58:07.0,2022-02-09,2022-03-01,12.00,2022-02-09,30.0,2022-02-09
3219,10293,1957-02-01,2.0,12.0,4,0.0,sc,1.0,2024-10-14,67.698836,...,0.001620,6.59,88.6,2025-08-06 10:07:33.0,2024-10-14,2024-11-12,4.67,2024-11-05,30.0,2024-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4284,492,1918-12-01,1.0,18.0,5,1.0,bl,3.0,2006-09-01,87.750856,...,0.031922,25.90,358.1,2024-12-18 09:58:07.0,2006-09-01,2006-05-31,30.00,2006-05-31,21.0,2006-12-07
4289,6459,1952-06-01,2.0,16.0,5,0.0,m48,1.0,2022-10-26,70.401095,...,0.001860,25.30,136.9,2024-12-18 09:58:07.0,2022-10-26,2022-07-20,3.33,2022-07-21,30.0,2022-07-21
4303,4809,1934-03-01,1.0,12.0,5,1.0,m48,2.0,2016-10-05,82.598220,...,0.010400,29.60,195.6,2024-12-18 09:58:07.0,2016-10-05,2016-07-15,28.00,2016-06-28,22.0,2016-06-28
4305,6600,1947-07-01,2.0,14.0,5,1.0,m12,3.0,2020-02-20,72.640657,...,0.019410,29.40,278.7,2024-12-18 09:58:07.0,2020-02-20,2019-11-13,NaN,2019-11-13,12.0,2019-11-13


In [967]:
closest_match.isna().sum()

RID                     0
PTDOB                   0
PTGENDER                0
PTEDUCAT                0
PTRACCAT                0
APOE41                  0
VISCODE2_g              0
DIAGNOSIS               0
EXAMDATE_x              0
AGE                     0
EXAMDATE_y              0
VISCODE_g               0
Intracranial Volume     0
Hippocampus             0
Amygdala                0
Temporal Lobe           0
Parietal Lobe           0
Frontal Lobe            0
Occipital Lobe          0
Cingulate Cortex        0
Insular Cortex          0
Basal Ganglia           0
Thalamus                0
date_diff               0
PHASE                   0
PTID                    0
VISCODE_d               0
VISCODE2_d              0
EXAMDATE                0
Primary                 0
Additive                0
pT217_F                 0
AB42_F                  0
AB40_F                  0
AB42_AB40_F             0
pT217_AB42_F            0
NfL_Q                   0
GFAP_Q                  0
update_stamp

In [968]:
closest_match.columns

Index(['RID', 'PTDOB', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT', 'APOE41',
       'VISCODE2_g', 'DIAGNOSIS', 'EXAMDATE_x', 'AGE', 'EXAMDATE_y',
       'VISCODE_g', 'Intracranial Volume', 'Hippocampus', 'Amygdala',
       'Temporal Lobe', 'Parietal Lobe', 'Frontal Lobe', 'Occipital Lobe',
       'Cingulate Cortex', 'Insular Cortex', 'Basal Ganglia', 'Thalamus',
       'date_diff', 'PHASE', 'PTID', 'VISCODE_d', 'VISCODE2_d', 'EXAMDATE',
       'Primary', 'Additive', 'pT217_F', 'AB42_F', 'AB40_F', 'AB42_AB40_F',
       'pT217_AB42_F', 'NfL_Q', 'GFAP_Q', 'update_stamp', 'EXAMDATE_g',
       'EXAMDATE_d', 'TOTAL13', 'VISDATE_c', 'MMSCORE', 'VISDATE_m'],
      dtype='object')

In [969]:
# cols = ['RID', 'EXAMDATE_x', 'AGE', 'PTEDUCAT', 'PTGENDER', 
#         # 'PTRACCAT', 
#         'APOE41', 'Intracranial Volume','Hippocampus', 'Amygdala', 'Temporal Lobe',
#         # 'Thalamus',
#         'pT217_F', 'NfL_Q', 'GFAP_Q', 'AB42_F', 'AB40_F', 'TOTAL13', 'MMSCORE']

# cols = ['RID', 'EXAMDATE_x',  'AGE','PTGENDER', 'PTEDUCAT',  'APOE41',

#        'Intracranial Volume', 'Hippocampus', 'Amygdala', 'Temporal Lobe',
# 'ABETA42', 'TAU', 'PTAU',
#        'TOTAL13', 'MMSCORE']


cols = ['RID',  'EXAMDATE_x','AGE', 'PTGENDER', 'PTEDUCAT', 'PTRACCAT', 'APOE41',
'Intracranial Volume', 'Hippocampus', 'Amygdala',
       'Temporal Lobe', 'pT217_F', 'AB42_F', 'AB40_F','NfL_Q', 'GFAP_Q', 
       'TOTAL13', 'MMSCORE']
closest_match = closest_match[cols]
closest_match = closest_match.dropna()

In [970]:
closest_match

,RID,EXAMDATE_x,AGE,PTGENDER,PTEDUCAT,PTRACCAT,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE
7,514,2011-06-08,85.850787,1.0,20.0,5,0.0,1.920956e+06,7463.0,2889.8,68258.0,0.111,36.19,374.31,47.80,282.8,11.33,28.0
3747,10391,2024-12-03,57.256674,2.0,16.0,5,0.0,1.507782e+06,7800.2,3389.0,82904.0,0.080,24.85,267.84,10.50,118.9,3.33,29.0
3748,10437,2025-01-13,68.950034,1.0,18.0,4,1.0,1.466964e+06,6552.1,2593.1,71448.0,1.009,21.09,293.29,21.30,194.7,20.67,24.0
1335,7041,2022-02-09,73.357974,2.0,20.0,5,1.0,1.405184e+06,7002.2,3142.3,70956.0,0.107,31.43,380.55,15.30,137.6,12.00,30.0
3219,10293,2024-10-14,67.698836,2.0,12.0,4,0.0,1.153377e+06,8074.2,3410.4,76404.0,0.043,26.60,327.09,6.59,88.6,4.67,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,6989,2021-10-26,68.320329,2.0,16.0,4,0.0,1.203910e+06,6308.6,2842.2,69979.0,0.288,24.49,354.87,21.00,194.7,13.33,24.0
4284,492,2006-09-01,87.750856,1.0,18.0,5,1.0,1.445368e+06,5938.7,2465.0,56418.0,0.852,26.69,375.42,25.90,358.1,30.00,21.0
4289,6459,2022-10-26,70.401095,2.0,16.0,5,0.0,1.278553e+06,8182.2,3465.5,77339.0,0.057,30.70,347.28,25.30,136.9,3.33,30.0
4303,4809,2016-10-05,82.598220,1.0,12.0,5,1.0,1.679190e+06,7176.0,2790.0,69741.0,0.294,28.26,328.46,29.60,195.6,28.00,22.0


In [971]:
# one hot encode PTRACCAT
closest_match = pd.get_dummies(closest_match, columns=['PTRACCAT'], drop_first=True)

In [972]:
closest_match.columns

Index(['RID', 'EXAMDATE_x', 'AGE', 'PTGENDER', 'PTEDUCAT', 'APOE41',
       'Intracranial Volume', 'Hippocampus', 'Amygdala', 'Temporal Lobe',
       'pT217_F', 'AB42_F', 'AB40_F', 'NfL_Q', 'GFAP_Q', 'TOTAL13', 'MMSCORE',
       'PTRACCAT_1|7', 'PTRACCAT_2', 'PTRACCAT_2|4', 'PTRACCAT_4',
       'PTRACCAT_5', 'PTRACCAT_6', 'PTRACCAT_7', 'PTRACCAT_9'],
      dtype='object')

In [973]:
# drop all rows for which PTRACCAT_2 or PTRACCAT_3 is True
closest_match = closest_match[~(closest_match['PTRACCAT_1|7'] | closest_match['PTRACCAT_2|4'] |  closest_match['PTRACCAT_9'] | closest_match['PTRACCAT_7'] | closest_match['PTRACCAT_6'] )]
closest_match = closest_match.drop(columns=['PTRACCAT_1|7', 'PTRACCAT_2|4', 'PTRACCAT_9', 'PTRACCAT_7', 'PTRACCAT_6' ])
closest_match

,RID,EXAMDATE_x,AGE,PTGENDER,PTEDUCAT,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE,PTRACCAT_2,PTRACCAT_4,PTRACCAT_5
7,514,2011-06-08,85.850787,1.0,20.0,0.0,1.920956e+06,7463.0,2889.8,68258.0,0.111,36.19,374.31,47.80,282.8,11.33,28.0,False,False,True
3747,10391,2024-12-03,57.256674,2.0,16.0,0.0,1.507782e+06,7800.2,3389.0,82904.0,0.080,24.85,267.84,10.50,118.9,3.33,29.0,False,False,True
3748,10437,2025-01-13,68.950034,1.0,18.0,1.0,1.466964e+06,6552.1,2593.1,71448.0,1.009,21.09,293.29,21.30,194.7,20.67,24.0,False,True,False
1335,7041,2022-02-09,73.357974,2.0,20.0,1.0,1.405184e+06,7002.2,3142.3,70956.0,0.107,31.43,380.55,15.30,137.6,12.00,30.0,False,False,True
3219,10293,2024-10-14,67.698836,2.0,12.0,0.0,1.153377e+06,8074.2,3410.4,76404.0,0.043,26.60,327.09,6.59,88.6,4.67,30.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,6989,2021-10-26,68.320329,2.0,16.0,0.0,1.203910e+06,6308.6,2842.2,69979.0,0.288,24.49,354.87,21.00,194.7,13.33,24.0,False,True,False
4284,492,2006-09-01,87.750856,1.0,18.0,1.0,1.445368e+06,5938.7,2465.0,56418.0,0.852,26.69,375.42,25.90,358.1,30.00,21.0,False,False,True
4289,6459,2022-10-26,70.401095,2.0,16.0,0.0,1.278553e+06,8182.2,3465.5,77339.0,0.057,30.70,347.28,25.30,136.9,3.33,30.0,False,False,True
4303,4809,2016-10-05,82.598220,1.0,12.0,1.0,1.679190e+06,7176.0,2790.0,69741.0,0.294,28.26,328.46,29.60,195.6,28.00,22.0,False,False,True


In [974]:
closest_match = closest_match.sort_values(by=['RID', 'EXAMDATE_x'])
closest_match_singular = closest_match.drop_duplicates(subset=['RID'])

In [975]:
cols = ['RID', 'EXAMDATE_x', 'AGE','PTEDUCAT', 'PTGENDER', 'PTRACCAT_2', 'PTRACCAT_4', 'PTRACCAT_5', 'APOE41',
       'Intracranial Volume', 'Hippocampus', 'Amygdala', 'Temporal Lobe',
       'pT217_F', 'AB42_F', 'AB40_F', 'NfL_Q', 'GFAP_Q', 'TOTAL13', 'MMSCORE']

In [976]:
closest_match = closest_match[cols]

In [977]:
closest_match

,RID,EXAMDATE_x,AGE,PTEDUCAT,PTGENDER,PTRACCAT_2,PTRACCAT_4,PTRACCAT_5,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE
3141,2,2005-09-29,74.496920,16.0,1.0,False,False,True,0.0,1.987463e+06,8211.8,3303.0,83471.0,0.199,27.92,324.12,14.9,94.6,18.67,28.0
2584,8,2005-09-30,84.583162,18.0,2.0,False,False,True,0.0,1.396097e+06,5671.2,2128.7,64287.0,0.152,27.90,296.53,26.6,141.1,7.00,28.0
1062,10,2005-11-10,73.943874,12.0,2.0,False,False,True,1.0,1.474304e+06,5582.9,2265.6,67978.0,0.472,24.44,332.15,26.5,194.3,24.33,24.0
344,10,2007-11-07,75.934292,12.0,2.0,False,False,True,1.0,1.473440e+06,5019.5,2077.0,61530.0,0.586,19.61,288.14,30.2,221.3,30.33,28.0
155,15,2005-10-18,80.960986,18.0,1.0,False,False,True,1.0,1.413627e+06,7441.3,2900.6,64040.0,0.291,32.80,331.38,29.8,106.3,9.00,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,10391,2024-12-03,57.256674,16.0,2.0,False,False,True,0.0,1.507782e+06,7800.2,3389.0,82904.0,0.080,24.85,267.84,10.5,118.9,3.33,29.0
3323,10406,2024-12-12,60.197125,18.0,2.0,False,True,False,1.0,1.236048e+06,6460.1,2893.6,63112.0,0.065,24.82,276.78,11.2,112.6,4.00,28.0
3829,10421,2025-01-06,74.600958,16.0,2.0,False,False,True,0.0,1.467947e+06,9068.6,3450.8,70435.0,0.135,35.61,342.46,11.6,232.2,11.33,29.0
3748,10437,2025-01-13,68.950034,18.0,1.0,False,True,False,1.0,1.466964e+06,6552.1,2593.1,71448.0,1.009,21.09,293.29,21.3,194.7,20.67,24.0


In [978]:
# find duplicate rows based on RID
duplicates = closest_match[closest_match.duplicated(subset=['RID'], keep=False)]
duplicates = duplicates.drop_duplicates(subset=['Hippocampus'])
duplicates

,RID,EXAMDATE_x,AGE,PTEDUCAT,PTGENDER,PTRACCAT_2,PTRACCAT_4,PTRACCAT_5,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE
1062,10,2005-11-10,73.943874,12.0,2.0,False,False,True,1.0,1.474304e+06,5582.9,2265.6,67978.0,0.472,24.44,332.15,26.5,194.30,24.33,24.0
344,10,2007-11-07,75.934292,12.0,2.0,False,False,True,1.0,1.473440e+06,5019.5,2077.0,61530.0,0.586,19.61,288.14,30.2,221.30,30.33,28.0
1258,42,2005-11-10,72.941821,18.0,1.0,False,False,True,0.0,1.520173e+06,4611.3,2149.1,64865.0,0.095,26.85,288.11,73.3,172.10,12.00,30.0
2668,42,2013-12-04,81.007529,18.0,1.0,False,False,True,0.0,1.503824e+06,4201.3,2264.1,53462.0,0.119,24.65,308.82,21.3,139.70,32.33,22.0
1383,53,2005-11-29,80.243669,12.0,1.0,False,False,True,0.0,1.748711e+06,6614.7,2215.3,65658.0,0.113,-4.00,-4.00,44.1,306.60,28.33,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,7055,2024-10-17,72.544832,16.0,1.0,False,True,False,0.0,1.429985e+06,8611.8,3469.2,78881.0,0.244,22.58,290.70,11.9,123.50,5.67,29.0
3664,7114,2022-10-13,65.694730,17.0,2.0,False,True,False,1.0,1.314348e+06,7782.2,3421.5,69797.0,0.033,25.11,256.33,10.1,80.40,0.67,30.0
2308,7114,2024-11-06,67.761807,17.0,2.0,False,True,False,1.0,1.294881e+06,7878.8,3304.0,70729.0,0.088,26.83,299.42,11.2,86.05,5.33,28.0
3246,10048,2024-02-05,78.847365,20.0,2.0,False,False,True,0.0,1.350819e+06,4935.1,1953.3,62595.0,0.226,30.37,360.70,24.0,363.40,19.67,29.0


In [979]:
# check = closest_match.drop_duplicates(subset=['Hippocampus'])
closest_match = closest_match.drop_duplicates(subset=['AB42_F', 'AB40_F', 'pT217_F', 'NfL_Q', 'GFAP_Q'])
closest_match

,RID,EXAMDATE_x,AGE,PTEDUCAT,PTGENDER,PTRACCAT_2,PTRACCAT_4,PTRACCAT_5,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE
3141,2,2005-09-29,74.496920,16.0,1.0,False,False,True,0.0,1.987463e+06,8211.8,3303.0,83471.0,0.199,27.92,324.12,14.9,94.6,18.67,28.0
2584,8,2005-09-30,84.583162,18.0,2.0,False,False,True,0.0,1.396097e+06,5671.2,2128.7,64287.0,0.152,27.90,296.53,26.6,141.1,7.00,28.0
1062,10,2005-11-10,73.943874,12.0,2.0,False,False,True,1.0,1.474304e+06,5582.9,2265.6,67978.0,0.472,24.44,332.15,26.5,194.3,24.33,24.0
344,10,2007-11-07,75.934292,12.0,2.0,False,False,True,1.0,1.473440e+06,5019.5,2077.0,61530.0,0.586,19.61,288.14,30.2,221.3,30.33,28.0
155,15,2005-10-18,80.960986,18.0,1.0,False,False,True,1.0,1.413627e+06,7441.3,2900.6,64040.0,0.291,32.80,331.38,29.8,106.3,9.00,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,10391,2024-12-03,57.256674,16.0,2.0,False,False,True,0.0,1.507782e+06,7800.2,3389.0,82904.0,0.080,24.85,267.84,10.5,118.9,3.33,29.0
3323,10406,2024-12-12,60.197125,18.0,2.0,False,True,False,1.0,1.236048e+06,6460.1,2893.6,63112.0,0.065,24.82,276.78,11.2,112.6,4.00,28.0
3829,10421,2025-01-06,74.600958,16.0,2.0,False,False,True,0.0,1.467947e+06,9068.6,3450.8,70435.0,0.135,35.61,342.46,11.6,232.2,11.33,29.0
3748,10437,2025-01-13,68.950034,18.0,1.0,False,True,False,1.0,1.466964e+06,6552.1,2593.1,71448.0,1.009,21.09,293.29,21.3,194.7,20.67,24.0


In [980]:
# for the blood dataframe, find any rows with identical values for each variable

bloods =  'AB42_F', 'AB40_F', 'pT217_F', 'NfL_Q', 'GFAP_Q'
# Find rows where (SampleID, Variable) is duplicated
duplicates = closest_match[closest_match.duplicated(subset=["RID", 'AB42_F', 'AB40_F', 'pT217_F', 'NfL_Q', 'GFAP_Q'], keep=False)]
duplicates


,RID,EXAMDATE_x,AGE,PTEDUCAT,PTGENDER,PTRACCAT_2,PTRACCAT_4,PTRACCAT_5,APOE41,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,pT217_F,AB42_F,AB40_F,NfL_Q,GFAP_Q,TOTAL13,MMSCORE


In [981]:
closest_match['RID'].nunique(), closest_match.shape

(807, (918, 20))

In [984]:
df.groupby("RID").size().max()

np.int64(2)

In [983]:
import numpy as np
import pandas as pd

# Sort by patient and time (important for alignment)
df = closest_match

# Get unique patients and max timepoints
patients = df["RID"].unique()
n_patients = len(patients)
max_timepoints = df.groupby("RID").size().max()

# Extract variables
variables = [c for c in df.columns if c not in ["RID", "EXAMDATE_x"]]
n_vars = len(variables)

# Initialize array with NaNs
arr = np.full((max_timepoints, n_patients, n_vars), np.nan)

# Fill the array
for i, pid in enumerate(patients):
    subdf = df[df["RID"] == pid].sort_values("EXAMDATE_x")
    values = subdf[variables].to_numpy()
    arr[:len(values), i, :] = values

print("Array shape:", arr.shape)
print(arr)


Array shape: (2, 807, 18)
[[[ 74.49691992  16.           1.         ...  94.6         18.67
    28.        ]
  [ 84.58316222  18.           2.         ... 141.1          7.
    28.        ]
  [ 73.94387406  12.           2.         ... 194.3         24.33
    24.        ]
  ...
  [ 74.60095825  16.           2.         ... 232.2         11.33
    29.        ]
  [ 68.95003422  18.           1.         ... 194.7         20.67
    24.        ]
  [ 73.57700205  18.           2.         ... 317.7         23.
    28.        ]]

 [[         nan          nan          nan ...          nan          nan
            nan]
  [         nan          nan          nan ...          nan          nan
            nan]
  [ 75.93429158  12.           2.         ... 221.3         30.33
    28.        ]
  ...
  [         nan          nan          nan ...          nan          nan
            nan]
  [         nan          nan          nan ...          nan          nan
            nan]
  [         nan          na

In [950]:
# check number of non-nans for each time point
for t in range(arr.shape[0]):
    non_nans = np.sum(~np.isnan(arr[t, :, 6]))
    print(f"Time point {t}: {non_nans} non-NaN entries")

Time point 0: 1524 non-NaN entries
Time point 1: 712 non-NaN entries
Time point 2: 249 non-NaN entries
Time point 3: 116 non-NaN entries
Time point 4: 53 non-NaN entries
Time point 5: 19 non-NaN entries
Time point 6: 4 non-NaN entries
Time point 7: 1 non-NaN entries


In [867]:
np.save('../data/merged_array_bloods_race.npy', arr)

In [ ]:
# closest_match_singular = closest_match_singular.iloc[:, 2:]
# closest_match_singular.to_csv('../data/merged_data_singular.csv', index=False)

In [406]:
# find how many RIDs appear 2 times
closest_match['RID'].value_counts().sort_values(ascending=False).value_counts()

count
1    716
2    112
Name: count, dtype: int64

In [407]:
duplicated_rids = closest_match[closest_match.duplicated(subset=['RID'], keep=False)]
duplicated_rids

,RID,EXAMDATE_x,AGE,PTEDUCAT,PTGENDER,PTRACCAT,APOE41,pT217_F,NfL_Q,GFAP_Q,AB42_F,AB40_F,Intracranial Volume,Hippocampus,Amygdala,Temporal Lobe,Thalamus,TOTAL13,MMSCORE
682,4039,2013-06-25,58.067077,19.0,1.0,5,1.0,1.405,24.6,255.0,23.88,295.64,1.397434e+06,6342.0,2647.4,50371.0,11752.2,64.33,13.0
720,7055,2024-10-17,72.544832,16.0,1.0,4,0.0,0.244,11.9,123.5,22.58,290.70,1.429985e+06,8611.8,3469.2,78881.0,11621.7,5.67,29.0
670,6145,2023-11-28,76.325804,16.0,2.0,5,0.0,0.175,21.0,209.9,36.32,312.56,1.434003e+06,6546.0,2900.0,77210.0,12098.6,4.33,29.0
723,853,2007-09-17,87.876797,16.0,1.0,5,0.0,0.206,23.0,312.4,23.95,296.36,1.536266e+06,5167.6,3261.5,64212.0,11157.4,30.00,21.0
759,78,2007-07-24,77.642710,18.0,2.0,5,1.0,1.264,47.7,504.1,27.23,305.59,1.652770e+06,6346.8,2386.1,64790.0,11846.8,72.00,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,4071,2011-08-23,84.807666,13.0,1.0,5,0.0,0.092,19.2,186.2,19.71,267.95,1.911021e+06,8342.9,3365.3,79131.0,14752.3,17.67,24.0
4419,4765,2017-08-29,81.242984,16.0,1.0,5,1.0,0.293,-5.0,-5.0,-4.00,-4.00,1.423040e+06,6455.6,1805.4,67607.0,11331.9,46.67,16.0
2339,6995,2021-12-06,58.929500,18.0,2.0,2,0.0,0.079,10.7,166.3,24.65,237.49,1.481552e+06,8180.9,3634.5,84222.0,16229.1,10.33,29.0
1835,6145,2018-03-14,70.617385,16.0,2.0,5,0.0,0.128,15.4,125.7,28.59,283.28,1.391597e+06,6376.5,2986.3,76307.0,11966.0,6.67,30.0
